In [10]:
import os

# Define the target directory (change yours)
target_directory = r"C:\Users\pablosal\Desktop\gbbai-semantic-kernel-nltosql"

# Check if the directory exists
if os.path.exists(target_directory):
    # Change the current working directory
    os.chdir(target_directory)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Directory {target_directory} does not exist.")

Directory changed to C:\Users\pablosal\Desktop\gbbai-semantic-kernel-nltosql


In [11]:
from dotenv import load_dotenv
import os
import asyncio
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from plugins.QueryDb import queryDb as plugin

kernel = sk.Kernel()

# Get AOAI settings from .env
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()

In [12]:
# Set the deployment name to the value of your chat model
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
azure_text_service = AzureChatCompletion(
    deployment_name=deployment, endpoint=endpoint, api_key=api_key
)
kernel.add_text_completion_service("azure_text_completion", azure_text_service)

# Immport NLP to SQL Plugin
plugins_directory = "plugins"
kernel.import_semantic_plugin_from_directory(plugins_directory, "nlpToSqlPlugin")
kernel.import_plugin(
    plugin.QueryDbPlugin(os.getenv("CONNECTION_STRING")), plugin_name="QueryDbPlugin"
)

KernelPlugin(name='QueryDbPlugin', description=None, functions={'query_db': KernelFunction(plugin_name='QueryDbPlugin', description='Query a database using a SQL query', name='query_db', is_semantic=False, stream_function=<bound method QueryDbPlugin.query_db of <plugins.QueryDb.queryDb.QueryDbPlugin object at 0x0000029AE4DCBE50>>, parameters=[ParameterView(name='input', description='SQL Query to be executed', default_value='', type_='string', required=False)], delegate_type=<DelegateTypes.InKernelContextOutString: 5>, function=<bound method QueryDbPlugin.query_db of <plugins.QueryDb.queryDb.QueryDbPlugin object at 0x0000029AE4DCBE50>>, plugins=KernelPluginCollection(plugins={'nlpToSqlPlugin': KernelPlugin(name='nlpToSqlPlugin', description=None, functions={'ConvertNLPToSQL': KernelFunction(plugin_name='nlpToSqlPlugin', description='Given a query and a list of possible intents, detect which intent the input matches', name='ConvertNLPToSQL', is_semantic=True, stream_function=<function Ke

In [13]:
async def create_plan(planner, input):
    return await planner.create_plan(goal=input)


async def invoke_plan(sequential_plan):
    return await sequential_plan.invoke()

In [14]:
import semantic_kernel as sk
from semantic_kernel.planning.sequential_planner import SequentialPlanner

In [15]:
planner = SequentialPlanner(kernel)

In [16]:
prompt = """Can you list the UserId, ProfileName, Score, Summary, and Recommendation for users whose
 HelpfulnessNumerator is greater than 15?"""

In [17]:
# Create a plan with the NLP input
ask = f"Create a SQL query according to the following request: {prompt} and query the database to get the result."

plan = await create_plan(planner, ask)

# Invoke the plan and get the result
result = await invoke_plan(plan)

In [22]:
result.result

"Response: It appears there was an error with the database query. The system could not find the columns 'value1', 'value2', and 'value3'. Please check the column names and try again."

In [19]:
print("/n")
print(f"User ASK: {prompt}")
print(f"Response: {result}")
print("/n")

# Print each step of the plan and its result
for index, step in enumerate(plan._steps):
    print("Step:", index)
    print("Description:", step.description)
    print("Function:", step.plugin_name + "." + step._function.name)
    if len(step._outputs) > 0:
        print("  Output:\n", str.replace(result[step._outputs[0]], "\n", "\n  "))
        print("\n\n")

/n
User ASK: Can you list the UserId, ProfileName, Score, Summary, and Recommendation for users whose
 HelpfulnessNumerator is greater than 15?
Response: Response: It appears there was an error with the database query. The system could not find the columns 'value1', 'value2', and 'value3'. Please check the column names and try again.
/n
Step: 0
Description: Given a query and a list of possible intents, detect which intent the input matches
Function: nlpToSqlPlugin.ConvertNLPToSQL
  Output:
 The provided schema does not contain the necessary fields (UserId, ProfileName, Score, Summary, Recommendation, HelpfulnessNumerator) to write the SQL query for the given objective. Please provide the correct schema.



Step: 1
Description: Convert SQL queries in any ANSI SQL dialect to a Transact-SQL dialect
Function: nlpToSqlPlugin.MakeSQLCompatible
  Output:
 INSERT INTO table_name (column1, column2, column3) VALUES (value1, value2, value3);



Step: 2
Description: Query a database using a SQL qu

In [21]:
step._outputs[0]

'RESULT__FINAL_ANSWER'

In [7]:
from src.aoai.azure_openai import AzureOpenAIManager
from src.prompts import get_intent_prompt

In [8]:
prompt_1 = """Can you list the UserId, ProfileName, Score, Summary, and Recommendation for users whose
 HelpfulnessNumerator is greater than 15?"""

prompt_2 = """lalalalala"""

prompt_3 = """How to start using the tool?"""

In [17]:
az_client = AzureOpenAIManager()

conversation_history = []

prompt_intent = get_intent_prompt(conversation_history, prompt_3)

intent = az_client.generate_chat_response(
    conversation_history=conversation_history,
    query=prompt_intent,
    system_message_content="You are an AI assistant designed to detect the intent from the user's query.",
    max_tokens=50,
)

2024-05-01 19:26:47,147 - micro - MainProcess - INFO     Sending request to OpenAI with query: 
    Based on the chat history and the user's most recent question, determine the intent of the query. The intent should be categorized into one of the following strings:

    how_to: Use this category for questions that seek guidance or instructions on how to perform specific tasks.
    database: Use this category for questions that request specific data or information from a database.
    how_to_database: Use this category for questions that both seek guidance on how to perform specific tasks and request specific data or information from a database.
    not_found: Use this category if the intent does not fit the categories above or cannot be clearly determined.
    Provide only the identified intent without any additional information.

    Take your time to analyze the user's query and the context provided in the chat history to determine the most appropriate intent.

    Chat History:
    

In [15]:
# %pip install azure-search-documents==11.4.0b10

In [18]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.models import RawVectorQuery
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Set up Azure Cognitive Search credentials
service_endpoint = os.getenv("AZURE_AI_SEARCH_SERVICE_ENDPOINT")
key = os.getenv("AZURE_SEARCH_ADMIN_KEY")
credential = AzureKeyCredential(key)

# Define the name of the Azure Search index
# This is the index where your data is stored in Azure Search
index_name = "query-dev-index"

# Set up the Azure Search client with the specified index
# This prepares the client to interact with the Azure Search service
search_client = SearchClient(service_endpoint, index_name, credential=credential)
search_query = SearchClient(service_endpoint, index_name, credential=credential)

In [19]:
search_query = "Identify players who played more than 100 games, have an OPS (On-base Plus Slugging) higher than .900, and have less than 10 errors in a season."
search_vector = az_client.generate_embedding(input_text=search_query)

In [20]:
if intent == "how_to":
    # hybrid retrieval + rerank
    r = search_client.search(
        search_query,
        top=5,
        vector_queries=[
            RawVectorQuery(vector=search_vector, k=50, fields="table_vector")
        ],
        query_type="semantic",
        semantic_configuration_name="query-index-semantic-config",
        query_language="en-us",
    )

    for doc in r:
        content = doc["table_content"].replace("\n", " ")[:1000]
        table_name = doc["table_name"]
        score = doc["@search.score"]
        reranker_score = doc["@search.reranker_score"]

        print(f"Table Name: {table_name}")
        print(f"Score: {score}")
        print(f"Reranker Score: {reranker_score}")
        print(f"Content: {content}")
        print("-" * 50)  # prints a separator for readability
elif intent == "database":
    # Handle 'database' intent
    print("Handling 'database' intent...")
    # Add your code here
elif intent == "how_to_database":
    # Handle 'how_to_database' intent
    print("Handling 'how_to_database' intent...")
    # Add your code here
else:
    # Handle 'not_found' intent
    print("Intent not found. Please try again.")
    # Add your code here

Table Name: detroit_tigers_baseball_stats
Score: 0.03306011110544205
Reranker Score: 2.504136085510254
Content: The provided schema is for a table named 'detroit_tigers_baseball_stats', which likely stores statistical data related to the players of the Detroit Tigers, a professional baseball team. The table has eight columns, each representing different characteristics or statistics related to a player.  1. 'name' (nvarchar): This column is used to store the name of the player. As the data type is 'nvarchar', it can contain both text and numbers, allowing for diverse names.  2. 'position' (nvarchar): This field stores the position of a player on the field. The 'nvarchar' data type indicates that this field can also contain alphanumeric characters, allowing for various positions like 'SS' (Short Stop), 'CF' (Center Field), etc.  3. 'Games_Played' (tinyint): This field records the number of games a player has played. The 'tinyint' data type indicates that this number is likely to be rela